# Prerequisites

## Modules

In [ ]:
# Modules are available in conda environment with name: icet
# conda activate icet

import ase
from ase.io import read as ASEread
from ase.io.vasp import write_vasp
from ase.db import connect
from ase.cell import Cell
from ase.neighborlist import NewPrimitiveNeighborList
from ase.build import make_supercell

import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import glob
import sys
import os
import random
import shutil

import icet
from icet import ClusterSpace, StructureContainer, ClusterExpansion
from trainstation import CrossValidationEstimator
from icet.tools import enumerate_structures
from icet.tools.structure_generation import generate_sqs_by_enumeration

try:
    import seaborn as sns
    sns.set_context('notebook')
except ImportError:
    print('sad')
    
import subprocess

import datetime

## Misc Functions

In [ ]:
# Stop message
def jupyter_stop(ErrorMessage="User-defined stop via jupyter_stop() function"):
    """
    User defined stop function, similar to exit(). Mostly for testing purpose or to 
    avoid overwriting of already generated data.
    """
    raise SystemExit(ErrorMessage)

## CE Functions

In [ ]:
# Basic setups

def get_fit_data(prim, chemical_symbols, cutoffs, energy_list, atoms_ref_list, outcar_list, position_tolerance, symprec, tol_positions):
    """
    Construct cluster space and structure container for the given cutoffs
    and return the fit matrix along with the target energies
    """
    # stepsize to print update of training:
    stepsize = int(0.1*len(outcar_list))
    
    # Collect the mapped structures
    mapped_structures = []
    
    # Set up Clusterspace
    cs = ClusterSpace(structure=prim,
                      cutoffs=cutoffs,
                      chemical_symbols=chemical_symbols,
                      position_tolerance=position_tolerance,
                      symprec=symprec)
    
    #print(cs)
    
    # Set up StructureContainer with the previsouly generated ClusterSpace
    sc = StructureContainer(cluster_space=cs)
    
    # Fill the StructureContainer
    for i, (outcar, E, at_ref) in enumerate(zip(outcar_list, energy_list, atoms_ref_list)):
        
        # print update of training
        if i % stepsize == 0:
            print(f"Computing structure {i} of {len(outcar_list)} ({i/len(outcar_list):.1%})   {datetime.datetime.now()}")
        
        
        # Read the OUTCAR [by default last step is used] and get energy
        #at     = ASEread(outcar)
        #total_energy = at.get_potential_energy() # total_energy = atoms.get_potential_energy(force_consistent=True)
        
        # Map the enumerated structure to the primitive cell, add it to cluster space with the energy of the properly relaxed system
        try:
            mapped_atoms, info = icet.tools.map_structure_to_reference(structure=at_ref, 
                                                             reference=prim, 
                                                             inert_species=["O"], 
                                                             tol_positions=tol_positions, 
                                                             suppress_warnings=False, 
                                                             assume_no_cell_relaxation=False)
            mapped_structures.append(mapped_atoms)

            sc.add_structure(structure=mapped_atoms,
                     properties={'Total Energy': E},
                     user_tag = outcar,
                     sanity_check=True,
                     )
        
        except ValueError as err:
            print(f"Mapping Error with {outcar}")
            print(f"Note: Possibly a different structure was used for the mapping!")
            print("Original Error Message:")
            print(err , "\n")
            

            
    print(f"len(cs) = {len(cs)}")
    
    return sc.get_fit_data(key='Total Energy'), mapped_structures


def get_A_y(prim, chemical_symbols, cutoffs, energy_list, atoms_ref_list, outcar_list, position_tolerance, symprec, tol_positions):
    return get_fit_data(prim, chemical_symbols, cutoffs, energy_list, atoms_ref_list, outcar_list, position_tolerance, symprec, tol_positions)



def get_row(cve, alpha=None):
    row = dict()
    row['rmse_validation'] = cve.rmse_validation
    row['rmse_train'] = cve.rmse_train
    row['BIC'] = cve.model.BIC
    row['n_parameters'] = cve.n_parameters
    row['n_nonzero_parameters'] = cve.n_nonzero_parameters
    
    if alpha != None:
        row['alpha'] = alpha
    
    return row


def train_ce(prim, chemical_symbols, cutoffs, energy_list, atoms_ref_list, outcar_list, fit_method, alpha=None):
    """
    Train a cluster expansion with the given cutoffs and return fit metrics of the obtained model.
    prim: ase atoms object, its the primitive structure that the CE lives on
    chemical_symbols: List of the possible atoms types on the different sites of prim
    cutoffs: cutoffs for the 2-body, 3-body, ... terms
    atoms_list: list of all the atoms objects to use for training/testing
    outcar_list : list with paths (strings) of the corresponding atoms objects
    fit_method examples with additional options (to be implemented at a later point): 
        fit_method='rfe'
        fit_method='ardr', threshold_lambda=4e5
        fit_method='ardr', line_scan=True
        fit_method='lasso'
        fit_method='least-squares'
    """
    A, y, mapped_structures = get_fit_data(prim, chemical_symbols, cutoffs, energy_list, atoms_ref_list, outcar_list)
    cve = CrossValidationEstimator((A, y), fit_method=fit_method, alpha=alpha, max_iter=50000,
                                   validation_method='shuffle-split', n_splits=10)
    cve.validate()
    cve.train()

    row = get_row(cve)
    
    return row, mapped_structures


## Reorder Atoms

In [ ]:
# S only in this list to 'trick' the structure enumeration
# S as extra Nickel
atomic_label2number = {"Li" :  3,
                       "O"  :  8,
                       "S"  : 16,
                       "Ni" : 28}

atomic_number2label = { 3 : "Li",
                        8 :  "O",
                       16 :  "S",
                       28 :  "Ni"}


def order_atoms(atoms,order=["Li","Ni","O"]):
    
    # get old positions and atomic numbers
    old_positions       = atoms.get_positions()
    old_atomic_number   = atoms.get_atomic_numbers()
    
    # create empty dict for all types
    atomic_pos_dict = {}
    for sym in order:
        atomic_pos_dict[sym] = []
    
    # append positions to dict 
    for num, pos in zip(old_atomic_number, old_positions):
        atomic_pos_dict[atomic_number2label[num]].append(pos)
    
    # put together the new ordered positions and atomic numbers
    new_positions = []
    new_atomic_numbers = []
    for sym in order:
        new_positions.extend(atomic_pos_dict[sym])
        new_atomic_numbers.extend( [ atomic_label2number[sym] ] * len(atomic_pos_dict[sym]) )
    
    # copy original atoms object and modify it
    copy_atoms = atoms.copy()
    copy_atoms.set_positions(new_positions)
    copy_atoms.set_atomic_numbers(new_atomic_numbers)
    
    return copy_atoms


# Collect data

In [ ]:
### Get the reference energies of LiNiO2 and NiO2 normed per unit cell
LiNiO2 = ASEread("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/CE_database_Marcel/02_enumerate_P21c_0-4fu/0001_finished_approved/run_final_approved/OUTCAR")
E_ref_LiNiO2_per_O2 = LiNiO2.get_potential_energy() / LiNiO2.get_chemical_symbols().count("O") * 2 #or per Ni in case of no extra Ni

NiO2   = ASEread("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/CE_database_Marcel/02_enumerate_P21c_0-4fu/0003_finished_approved/run_final_approved/OUTCAR")
E_ref_NiO2_per_O2   = NiO2.get_potential_energy() / NiO2.get_chemical_symbols().count("O") * 2 #or per Ni in case of no extra Ni

## Own enumerated structures based on P21/c

In [ ]:
#Li verteilung ohne trans
atoms_for_training_from_own_enumerated_structures = []
H_o_M_for_training_from_own_enumerated_structures = []

# Get all the outcars of interest
outcars_for_training_from_own_enumerated_structures= sorted(glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/CE_database_Marcel/02_enumerate_P21c_0-4fu/0*_finished_approved/run_final_approved/OUTCAR"))

# Iterate over them
for outcar in outcars_for_training_from_own_enumerated_structures:
    
    # get atoms object
    atoms = ASEread(outcar, index=":")
    
    # Compute total heat of mixing 
    Li_count = atoms[-1].get_chemical_symbols().count("Li")
    O_count  = atoms[-1].get_chemical_symbols().count("O")
    H_o_M    = atoms[-1].get_potential_energy() - Li_count * E_ref_LiNiO2_per_O2 - (O_count/2-Li_count) * E_ref_NiO2_per_O2

    # Append data...     
    # ... but to make mapping easier take the originally generated structures instead of the relaxed ones
    ref = "/".join(outcar.split("/")[:-2]) + "/POSCAR_enumerated"
    atoms_for_training_from_own_enumerated_structures.append(ASEread(ref))
    #refactor to HOM per Atom
    H_o_M_for_training_from_own_enumerated_structures.append( H_o_M / len(atoms[-1]) )



## Markus low energy CE data

In [ ]:
# The following code takes Markus' relaxed structures and rotates them around the ccartesian z axis to fit the primitive structure we are using
# Does not need to be run again
jupyter_stop(ErrorMessage="Structures have already been rotated, no need to run again.")

# Needs rotation to be properly mapped
READMEs = glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/CE_database_Marcel/03_Markus_approved_low_energy_data/*/README_original_path_from_Markus")

# Define the ovitos executable and script to do some transformation for markus' data
ovitos_executable = "/nfshome/sadowski/Programs/ovito-pro-3.8.4-x86_64/bin/ovitos"
ovitos_script     = "/nfshome/sadowski/work/LiNiO2_Sabrina/37_CE_for_Li_diffusion/00_notebooks/ovito_AffTrans_Markus_data.py"


for README in READMEs:
    
    # There were two problematic ones, for which the mapping did not work properly because the cell
    # got cheared too much during relaxation. Only for these, take the originally created structure instead
    # of the relaxed CONTCAR
    if "re-relax_Markus038_finished" in README or "re-relax_Markus115_finished" in README:
        
        # Read the README to get the original path
        with open(README, "r") as f:
            line = f.readlines()[0]
        orig_POSCAR_path = "/".join(line.split("/")[0:-1]) + "/run01/POSCAR" 
        atoms = ASEread(orig_POSCAR_path)

        # write out the relaxed structure
        write_vasp(orig_POSCAR_path + "_original.vasp", atoms)

        # Use ovito to rotate the relaxed structures
        subprocess.run([ovitos_executable, ovitos_script, orig_POSCAR_path + "_original.vasp", orig_POSCAR_path + "_rotated.vasp"])
        
        
    else:
        # Get the final CONTCAR
        contcar = README.replace("README_original_path_from_Markus","run_final/CONTCAR")
        atoms = ASEread(contcar)

        # write out the relaxed structure
        write_vasp(README.replace("README_original_path_from_Markus","run_final/CONTCAR.vasp"), atoms)

        # Use ovito to rotate the relaxed structures
        subprocess.run([ovitos_executable, ovitos_script, README.replace("README_original_path_from_Markus","run_final/CONTCAR.vasp"), README.replace("README_original_path_from_Markus","run_final/CONTCAR_rotated.vasp")])

In [ ]:
#Li verteilung ohne trans
# Compare Markus last step energy and volume with the ones re-relaxed from me
# To this end, use the README files where 

atoms_for_training_from_Markus_low_energy_structures   = []
H_o_M_for_training_from_Markus_low_energy_structures   = []
outcars_for_training_from_Markus_low_energy_structures = []

READMEs = glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/CE_database_Marcel/03_Markus_approved_low_energy_data/*/README_original_path_from_Markus")

for README in READMEs:
    
    if os.path.isdir(README.replace("README_original_path_from_Markus","run_final")):
    
        # get the transformed CONTCAR to enable correct mapped to our prim structure later
        # Only for the 2 structures that made problems, take the original (rotated) POSCAR to enable mapping later
        if "re-relax_Markus038_finished" in README or "re-relax_Markus115_finished" in README:
            with open(README, "r") as f:
                line = f.readlines()[0]
            transformed_contcar = "/".join(line.split("/")[0:-1]) + "/run01/POSCAR_rotated.vasp" 
        else:
            transformed_contcar = README.replace("README_original_path_from_Markus","run_final/CONTCAR_rotated.vasp")
        atoms_transformed_contcar = ASEread(transformed_contcar)
        atoms_for_training_from_Markus_low_energy_structures.append(atoms_transformed_contcar)
        
        # get the outcar from the relaxation to get the energy
        outcar = README.replace("README_original_path_from_Markus","run_final/OUTCAR")
        outcars_for_training_from_Markus_low_energy_structures.append(outcar)
        atoms = ASEread(outcar, index=":")

        # Compute heat of mixing and per atom
        Li_count = atoms[-1].get_chemical_symbols().count("Li")
        O_count  = atoms[-1].get_chemical_symbols().count("O")
        H_o_M    = atoms[-1].get_potential_energy() - Li_count * E_ref_LiNiO2_per_O2 - (O_count/2-Li_count) * E_ref_NiO2_per_O2    
        H_o_M_for_training_from_Markus_low_energy_structures.append( H_o_M / len(atoms[-1]) )
        

## NEB initial and final images (without Ni_Li)

In [ ]:
#Li verteilung ohne trans
atoms_for_training_NEB_initial_and_final_images = []
H_o_M_for_training_NEB_initial_and_final_images = []

# Find the ordered ones from 0250, 0500 and 0750 first
outcars_for_training_NEB_initial_and_final_images  = glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0250/image*/02_scan/*final/OUTCAR", recursive=True)
outcars_for_training_NEB_initial_and_final_images += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0500/image*/02_scan/*final/OUTCAR", recursive=True)
outcars_for_training_NEB_initial_and_final_images += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0750/image*/02_scan/*final/OUTCAR", recursive=True)

# and the ones from the random structures
outcars_for_training_NEB_initial_and_final_images += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0*random*/01_initial_structure/02_scan/*final/OUTCAR", recursive=True)
outcars_for_training_NEB_initial_and_final_images += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0*random*/02_odh/image*/02_scan/*final/OUTCAR", recursive=True)
outcars_for_training_NEB_initial_and_final_images += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0*random*/03_tsh/image*/02_scan/*final/OUTCAR", recursive=True)
outcars_for_training_NEB_initial_and_final_images += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0*random*/04_double_tsh/image*/02_scan/*final/OUTCAR", recursive=True)

# Iterate over OUTCARs
for OUTCAR in outcars_for_training_NEB_initial_and_final_images:
    
    # get the atoms object
    atoms = ASEread(OUTCAR, index=":") 
    
    # Compute heat of mixing and per atom
    Li_count = atoms[-1].get_chemical_symbols().count("Li")
    O_count  = atoms[-1].get_chemical_symbols().count("O")
    H_o_M    = atoms[-1].get_potential_energy() - Li_count * E_ref_LiNiO2_per_O2 - (O_count/2-Li_count) * E_ref_NiO2_per_O2
        
    # append them to the lists
    atoms_for_training_NEB_initial_and_final_images.append(atoms[-1])
    H_o_M_for_training_NEB_initial_and_final_images.append( H_o_M / len(atoms[-1]) )

## NEB transition states

In [ ]:
#Li-trans
fig, ax = plt.subplots()

atoms_for_training_NEB_transition_states = []
H_o_M_for_training_NEB_transition_states = []
paths_for_training_NEB_transition_states = []

# the ones generated manually (0250, 0500, 0750)
paths_for_training_NEB_transition_states  = glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0250/NEB_*_finished/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0500/NEB_*_finished/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0750/NEB_*_finished/run_final")

# the random ones
paths_for_training_NEB_transition_states += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0*random*/02_odh/NEB_*/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0*random*/03_tsh/NEB_*/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel/0*random*/04_double_tsh/NEB_*/run_final")

# Iterate over all run_final folders
for path in paths_for_training_NEB_transition_states:
    
    # Check the energy along the path. Use initial and final energies from the corresponding relaxed structures + the last steps of the 
    # optimized intermediate images
    energies = []
    energies.append(ASEread(path.replace("run_final", "OUTCAR_initial_image")).get_potential_energy())
    for i in ["01", "02", "03", "04", "05"]:
        energies.append(ASEread(f"{path}/{i}/OUTCAR").get_potential_energy())
    energies.append(ASEread(path.replace("run_final", "OUTCAR_final_image")).get_potential_energy())
    ax.plot([0,1,2,3,4,5,6], np.array(energies)-energies[0])
    
    # For "Proper" paths, there should be maximum in energy !between! initial and final paths... ignore those where this is not the case
    index_highest_energy = energies.index(max(energies))
    
    if index_highest_energy == 0 or index_highest_energy == 6:
        print(f"Ignore {path}\n  ---> image {index_highest_energy} has highest energy!")
    
    else:    
        # get the interpolated middle points of the initially created, straight odh-type path to be used as ideal position for the CE training
        ideal_TS_structure_file = glob.glob(path.replace("run_final", "run01*/03/POSCAR_orig_linear_interpolation"))
        if len(ideal_TS_structure_file) == 0:                # For ODH-type jumps there is no POSCAR_orig_linear_interpolation
            ideal_TS_structure_file = glob.glob(path.replace("run_final", "run01*/03/POSCAR"))
        ideal_TS_atoms = ASEread(ideal_TS_structure_file[0])
     
        atoms_for_training_NEB_transition_states.append(ideal_TS_atoms)
        
        # Compute heat of mixing per atom and append to list
        Li_count = ideal_TS_atoms.get_chemical_symbols().count("Li")
        O_count  = ideal_TS_atoms.get_chemical_symbols().count("O")
        H_o_M    = max(energies) - Li_count * E_ref_LiNiO2_per_O2 - (O_count/2-Li_count) * E_ref_NiO2_per_O2
        H_o_M_for_training_NEB_transition_states.append( H_o_M / len(ideal_TS_atoms) )
        
    


## Combine data

In [ ]:
# combine the structures 
train_structures = ( atoms_for_training_from_own_enumerated_structures 
                    + atoms_for_training_from_Markus_low_energy_structures
                    + atoms_for_training_NEB_initial_and_final_images 
                    + atoms_for_training_NEB_transition_states 
             )

# ... and energies
train_H_o_M      = ( H_o_M_for_training_from_own_enumerated_structures 
                     + H_o_M_for_training_from_Markus_low_energy_structures
                     + H_o_M_for_training_NEB_initial_and_final_images 
                     + H_o_M_for_training_NEB_transition_states 
             )

# to be able to retrieve problematic files, keep the paths
file_location = ( outcars_for_training_from_own_enumerated_structures 
                + outcars_for_training_from_Markus_low_energy_structures 
                + outcars_for_training_NEB_initial_and_final_images 
                + paths_for_training_NEB_transition_states
                )

# Fitting of just the Li sublattice

In [ ]:
# Read R-3m model of LiNiO2 in R-3m symmetry with transition states
prim_without_TS = ASEread("/nfshome/sadowski/work/LiNiO2_Sabrina/37_CE_for_Li_diffusion/00_LNO_R-3m.vasp")

print(prim_without_TS)
print(prim_without_TS.get_chemical_symbols())

In [ ]:
# Assign chemical symbols
chemical_symbols_without_TS= [['Li', 'X'],   # Li sublattice will contain: Li and Vacancies (=X), later also Ni
                    ['Ni'],       # Ni sublattice will not be changed
                    ['O'],        # O  sublattice will not be changed
                    ['O']]        

In [ ]:
# get the fitting data
cutoffs = [8.57, 8.09, 6.42]
position_tolerance = 0.01
symprec = 0.01
tol_positions=0.05

(A_without_TS, y_without_TS), mapped_structures_without_TS = get_A_y(prim=prim_without_TS, 
                                   chemical_symbols=chemical_symbols_without_TS, 
                                   cutoffs=cutoffs,                
                                   energy_list = train_H_o_M, 
                                   atoms_ref_list = train_structures, 
                                   outcar_list = file_location,
                                   position_tolerance=position_tolerance,
                                   symprec=symprec,
                                   tol_positions=tol_positions)

# Markus:  8.5966 Å, 8.1068 Å and 6.4169 Å

# Concerning pair cutoff within one Li layer
# Neighbor       distance           how many within layer      summed total per particle
#                                                               including Li layers above/below
# 1st neighbor at 2.8429                N= 6                         N =  6 (only intralayer bonds)
# 2nd neighbor at 4.92404               N= 6                         N = 12 (only intralayer bonds)
# 3rd neighbor at 5.68579 (=2*2.8429)   N= 6                         N = 24 (first bonds to other layers. 3 bonds to layer above and 3 to layer below)  
# 4th neighber at 7.5216                N=12                         N = 54 
# 5th neighbor at 8.52869 (=3*2.8429)   N= 6                         N = 78 

# Note: 4th cutoff here is just slightly smaller than 4th interlayer cutoff.
# Note: 5th cutoff here is just slightly smaller than 6th interlayer cutoff.


# Concerning cutoff for interlayer bonds:
# 1st interlayerbonds     at  4.99272   N= 3 (to both above and below)   Total interlayer summed up= 6
# 2nd interlayerbonds     at  5.74537   N= 3 (to both above and below)   Total interlayer summed up= 12
# 3rd interlayerbonds     at  6.41025   N= 6 (to both above and below)   Total interlayer summed up= 24
# 4th interlayerbonds     at  7.56673   N= 6 (to both above and below)   Total interlayer summed up= 36
# 5th interlayerbonds     at  8.08316   N= 3 (to both above and below)   Total interlayer summed up= 42
# 6th interlayerbonds     at  8.56852   N= 6 (to both above and below)   Total interlayer summed up= 54

#Total number of Li-Li Bonds as function of cutoff (inter/intra):
# 2.84    0  ( 0/ 0)
# 2.85    6  ( 6/ 0)
# 4.93   12  (12/ 0)
# 5.00   18  (12/ 6)
# 5.69   24  (18/ 6)
# 5.75   30  (18/12)  
# 6.42   42  (18/24)  3rd    3rd
# 7.53   54  (30/24)  2nd
# 7.57   66  (30/36)
# 8.09   72  (30/42)         2nd
# 8.53   78  (36/42)
# 8.57   90  (36/54)  1st    1st


In [ ]:
# scan ARDR
lambda_values = [1000]
records = []
for lam in lambda_values:
    cve = CrossValidationEstimator((A_without_TS, y_without_TS), fit_method='ardr', threshold_lambda=lam)
    cve.validate()
    cve.train()
    row = get_row(cve)
    row['threshold_lambda'] = lam
    records.append(row)
df_ardr = pd.DataFrame(records)
print(row)

# Set up Clusterspace
cs = ClusterSpace(structure=prim_without_TS,
                      cutoffs=cutoffs,
                      chemical_symbols=chemical_symbols_without_TS,
                      position_tolerance=position_tolerance,
                      symprec=symprec)


ce = ClusterExpansion(cluster_space=cs, parameters=cve.parameters, metadata=cve.summary)
print(ce)
ce.write('/nfshome/winkelmann/ARL/tmp/mixing_energy_no_TS.ce')
    

In [ ]:
# Read the previously outputted CE and set up a dictionary with data to be plotted later
ce = ClusterExpansion.read('/nfshome/winkelmann/ARL/tmp/mixing_energy_no_TS.ce')
write_vasp("/nfshome/winkelmann/ARL/tmp/real_prim.vasp", ce.primitive_structure)

# Store stuff for use later
data = {'concentration': [], 'reference_energy': [], 'predicted_energy': [], 'file_location': []}

# Go trough all the data
for outcar, mapped_structure, h_o_m, location in zip(file_location, mapped_structures_without_TS, train_H_o_M, file_location):
    
    try:
        # Compute Li concentration
        data['concentration'].append(mapped_structure.get_chemical_symbols().count("Li")/(mapped_structure.get_chemical_symbols().count("O")/2))

        # Add original energy to dictthe factor of 1e3 serves to convert from eV/atom to meV/atom
        data['reference_energy'].append(1e3 * h_o_m)

        # use the mapped structures to predict energy
        data['predicted_energy'].append(1e3 * ce.predict(mapped_structure))
        
        # keep the file location to allow parsing
        data['file_location'].append(location)
    
    # Catch errors in case something goes wrong
    except Exception as err:
        print(f"Problems with {outcar}")
        print(f"Original Error Message:\n {err}\n")
        

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(18, 18))

### ax1 = Full heat of mixing plot

ax1.set_xlabel(r'x in Li$_x$NiO2')
ax1.set_ylabel(r'Mixing energy (meV/atom)')
ax1.set_xlim([0, 1])
ax1.set_ylim([-40, 30])

ax1.scatter(data['concentration'], data['reference_energy'],
           marker='o', label='reference')
ax1.scatter(data['concentration'], data['predicted_energy'],
           marker='x', label='CE prediction')

ax1.legend()
ax1.set_title("Full CE plot")



### ax2 = Heat of mixing plot of my own structures

ax2.set_xlabel(r'x in Li$_x$NiO2')
ax2.set_ylabel(r'Mixing energy (meV/atom)')
ax2.set_xlim([0, 1])
ax2.set_ylim([-40, 30])

# Take only the ones we are interested here
concentration    = []
referende_energy = []
predicted_energy = []

for c, ref , pred, location in zip(data["concentration"], data["reference_energy"], data["predicted_energy"], data["file_location"]):
    if "02_enumerate_P21c_0-4fu" in location:
        concentration.append(c)
        referende_energy.append(ref)
        predicted_energy.append(pred)

ax2.scatter(concentration, referende_energy,
           marker='o', label='reference')
ax2.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction')

ax2.legend()
ax2.set_title("Marcel's enumerated data")



### ax3 = Heat of mixing plot of Markus's re-relaxed_structures

ax3.set_xlabel(r'x in Li$_x$NiO2')
ax3.set_ylabel(r'Mixing energy (meV/atom)')
ax3.set_xlim([0, 1])
ax3.set_ylim([-40, 30])

# Take only the ones we are interested here
concentration    = []
referende_energy = []
predicted_energy = []

for c, ref , pred, location in zip(data["concentration"], data["reference_energy"], data["predicted_energy"], data["file_location"]):
    if "re-relax_Markus" in location:
        concentration.append(c)
        referende_energy.append(ref)
        predicted_energy.append(pred)

ax3.scatter(concentration, referende_energy,
           marker='o', label='reference')
ax3.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction')

ax3.legend()
ax3.set_title("Markus' re-relaxed structures")



### ax4 = Heat of mixing plot of the initial and final NEB images

ax4.set_xlabel(r'x in Li$_x$NiO2')
ax4.set_ylabel(r'Mixing energy (meV/atom)')
ax4.set_xlim([0, 1])
ax4.set_ylim([-40, 30])

# Take only the ones we are interested here
concentration    = []
referende_energy = []
predicted_energy = []

for c, ref , pred, location in zip(data["concentration"], data["reference_energy"], data["predicted_energy"], data["file_location"]):
    if "01_initial_structure" in location \
    or "02_odh/image"         in location \
    or "03_tsh/image"         in location \
    or "04_doube_tsh/image"   in location \
    or "0250/image"           in location \
    or "0500/image"           in location \
    or "0750/image"           in location:
        concentration.append(c)
        referende_energy.append(ref)
        predicted_energy.append(pred)

ax4.scatter(concentration, referende_energy,
           marker='o', label='reference')
ax4.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction')

ax4.legend()
ax4.set_title("Initial and final NEB images")




plt.savefig('/nfshome/winkelmann/ARL/tmp/mixing_energy_comparison_no_TS.png', bbox_inches='tight')

# First fitting for testing purposes of python implementation (no Ni_Li, but with transition states)

In [ ]:
# Read R-3m model of LiNiO2 in R-3m symmetry with transition states
prim = ASEread("/nfshome/sadowski/work/LiNiO2_Sabrina/37_CE_for_Li_diffusion/00_LNO_R-3m_1fu_transformed_with_transition_states.vasp")

print(prim)
print(prim.get_chemical_symbols())

In [ ]:
# Assign chemical symbols
chemical_symbols= [['Li', 'X'],   # Li sublattice will contain: Li and Vacancies (=X), later also Ni
                    ['Ni'],       # Ni sublattice will not be changed
                    ['O'],        # O  sublattice will not be changed
                    ['O'],
                    ['Li','X'],   # Transition state sites
                    ['Li','X'],
                    ['Li','X'],]        

In [ ]:
# get the fitting data
cutoffs = [8.57, 7.53, 6.42]
position_tolerance = 0.01
symprec = 0.01
tol_positions=0.05

(A, y), mapped_structures = get_A_y(prim=prim, 
                                   chemical_symbols=chemical_symbols, 
                                   cutoffs=cutoffs,                
                                   energy_list = train_H_o_M, 
                                   atoms_ref_list = train_structures, 
                                   outcar_list = file_location,
                                   position_tolerance=position_tolerance,
                                   symprec=symprec,
                                   tol_positions=tol_positions)

# Markus:  8.5966 Å, 8.1068 Å and 6.4169 Å

# Concerning pair cutoff within one Li layer
# Neighbor       distance           how many within layer      summed total per particle
#                                                               including Li layers above/below
# 1st neighbor at 2.8429                N= 6                         N =  6 (only intralayer bonds)
# 2nd neighbor at 4.92404               N= 6                         N = 12 (only intralayer bonds)
# 3rd neighbor at 5.68579 (=2*2.8429)   N= 6                         N = 24 (first bonds to other layers. 3 bonds to layer above and 3 to layer below)  
# 4th neighber at 7.5216                N=12                         N = 54 
# 5th neighbor at 8.52869 (=3*2.8429)   N= 6                         N = 78 

# Note: 4th cutoff here is just slightly smaller than 4th interlayer cutoff.
# Note: 5th cutoff here is just slightly smaller than 6th interlayer cutoff.


# Concerning cutoff for interlayer bonds:
# 1st interlayerbonds     at  4.99272   N= 3 (to both above and below)   Total interlayer summed up= 6
# 2nd interlayerbonds     at  5.74537   N= 3 (to both above and below)   Total interlayer summed up= 12
# 3rd interlayerbonds     at  6.41025   N= 6 (to both above and below)   Total interlayer summed up= 24
# 4th interlayerbonds     at  7.56673   N= 6 (to both above and below)   Total interlayer summed up= 36
# 5th interlayerbonds     at  8.08316   N= 3 (to both above and below)   Total interlayer summed up= 42
# 6th interlayerbonds     at  8.56852   N= 6 (to both above and below)   Total interlayer summed up= 54

#Total number of Li-Li Bonds as function of cutoff (inter/intra):
# 2.84    0  ( 0/ 0)
# 2.85    6  ( 6/ 0)
# 4.93   12  (12/ 0)
# 5.00   18  (12/ 6)
# 5.69   24  (18/ 6)
# 5.75   30  (18/12)  
# 6.42   42  (18/24)  3rd    3rd
# 7.53   54  (30/24)  2nd
# 7.57   66  (30/36)
# 8.09   72  (30/42)         2nd
# 8.53   78  (36/42)
# 8.57   90  (36/54)  1st    1st


In [ ]:
# scan ARDR
lambda_values = [1000]
records = []
for lam in lambda_values:
    cve = CrossValidationEstimator((A, y), fit_method='ardr', threshold_lambda=lam)
    cve.validate()
    cve.train()
    row = get_row(cve)
    row['threshold_lambda'] = lam
    records.append(row)
df_ardr = pd.DataFrame(records)
print(row)

# Set up Clusterspace
cs = ClusterSpace(structure=prim,
                      cutoffs=cutoffs,
                      chemical_symbols=chemical_symbols,
                      position_tolerance=position_tolerance,
                      symprec=symprec)


ce = ClusterExpansion(cluster_space=cs, parameters=cve.parameters, metadata=cve.summary)
print(ce)
ce.write('/nfshome/winkelmann/ARL/tmp/mixing_energy.ce')
    

In [ ]:
# Read the previously outputted CE and set up a dictionary with data to be plotted later
ce = ClusterExpansion.read('/nfshome/winkelmann/ARL/tmp/mixing_energy.ce')
write_vasp("/nfshome/winkelmann/ARL/tmp/real_prim.vasp", ce.primitive_structure)

# Store stuff for use later
data = {'concentration': [], 'reference_energy': [], 'predicted_energy': [], 'file_location': []}

# Go trough all the data
for outcar, mapped_structure, h_o_m, location in zip(file_location, mapped_structures, train_H_o_M, file_location):
    
    try:
        # Compute Li concentration
        data['concentration'].append(mapped_structure.get_chemical_symbols().count("Li")/(mapped_structure.get_chemical_symbols().count("O")/2))

        # Add original energy to dictthe factor of 1e3 serves to convert from eV/atom to meV/atom
        data['reference_energy'].append(1e3 * h_o_m)

        # use the mapped structures to predict energy
        data['predicted_energy'].append(1e3 * ce.predict(mapped_structure))
        
        # keep the file location to allow parsing
        data['file_location'].append(location)
    
    # Catch errors in case something goes wrong
    except Exception as err:
        print(f"Problems with {outcar}")
        print(f"Original Error Message:\n {err}\n")
        

In [ ]:
# Retrieve the energy barriers and the corresponding predictons
barrier_concentrations  = []
ref_frontjump_barriers  = []
ref_backjump_barriers   = []
pred_frontjump_barriers = []
pred_backjump_barriers  = []

# Iterate over all run_final folders
for path in paths_for_training_NEB_transition_states:
        
    # Check the energy along the path. Use initial and final energies from the corresponding relaxed structures + the last steps of the 
    # optimized intermediate images
    energies = []
    initial_image = ASEread(path.replace("run_final", "OUTCAR_initial_image"))
    energies.append(initial_image.get_potential_energy())
    for i in ["01", "02", "03", "04", "05"]:
        energies.append(ASEread(f"{path}/{i}/OUTCAR").get_potential_energy())
    final_image = ASEread(path.replace("run_final", "OUTCAR_final_image"))
    energies.append(final_image.get_potential_energy())
    
    # For "Proper" paths, there should be maximum in energy !between! initial and final paths... ignore those where this is not the case
    index_highest_energy = energies.index(max(energies))
    
    if index_highest_energy == 0 or index_highest_energy == 6:
        print(f"Ignore {path}\n  ---> image {index_highest_energy} has highest energy!")
    
    else:
        # Compute reference frontjump and backjump barrier
        ref_frontjump_barriers.append(max(energies) - energies[0])
        ref_backjump_barriers.append(max(energies) - energies[-1])
        
        # Compute the predicted barriers...
        # get the interpolated middle points of the initially created, straight odh-type path to be used as ideal position for the CE training
        ideal_TS_structure_file = glob.glob(path.replace("run_final", "run01*/03/POSCAR_orig_linear_interpolation"))
        if len(ideal_TS_structure_file) == 0:                # For ODH-type jumps there is no POSCAR_orig_linear_interpolation
            ideal_TS_structure_file = glob.glob(path.replace("run_final", "run01*/03/POSCAR"))
        ideal_TS_atoms = ASEread(ideal_TS_structure_file[0])
        
        # Map the initial, final and TS state:        
        initial_mapped_atoms, info = icet.tools.map_structure_to_reference(structure=initial_image, 
                                                             reference=prim, 
                                                             inert_species=["O"], 
                                                             tol_positions=0.05, 
                                                             suppress_warnings=False, 
                                                             assume_no_cell_relaxation=False)
                          
        final_mapped_atoms,   info = icet.tools.map_structure_to_reference(structure=final_image, 
                                                             reference=prim, 
                                                             inert_species=["O"], 
                                                             tol_positions=0.05, 
                                                             suppress_warnings=False, 
                                                             assume_no_cell_relaxation=False)
        
        TS_mapped_atoms, info = icet.tools.map_structure_to_reference(structure=ideal_TS_atoms, 
                                                             reference=prim, 
                                                             inert_species=["O"], 
                                                             tol_positions=0.05, 
                                                             suppress_warnings=False, 
                                                             assume_no_cell_relaxation=False)
        
        # Fromt their predicted <<<heat of mixing>>>, get the energies and from those get the barriers:
        N_atoms = len(ideal_TS_atoms)
        Li_count = ideal_TS_atoms.get_chemical_symbols().count("Li")
        O_count  = ideal_TS_atoms.get_chemical_symbols().count("O")
                          
        pred_E_init  = ce.predict(initial_mapped_atoms) * N_atoms + Li_count * E_ref_LiNiO2_per_O2 + (O_count/2-Li_count) * E_ref_NiO2_per_O2
        pred_E_final = ce.predict(final_mapped_atoms)   * N_atoms + Li_count * E_ref_LiNiO2_per_O2 + (O_count/2-Li_count) * E_ref_NiO2_per_O2
        pred_E_TS    = ce.predict(TS_mapped_atoms)      * N_atoms + Li_count * E_ref_LiNiO2_per_O2 + (O_count/2-Li_count) * E_ref_NiO2_per_O2
                        
        pred_frontjump_barriers.append(pred_E_TS - pred_E_init)
        pred_backjump_barriers.append( pred_E_TS - pred_E_final)                  
                        
        barrier_concentrations.append(Li_count/(O_count/2))

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2, figsize=(18, 25))

### ax1 = Full heat of mixing plot

ax1.set_xlabel(r'x in Li$_x$NiO2')
ax1.set_ylabel(r'Mixing energy (meV/atom)')
ax1.set_xlim([0, 1])
ax1.set_ylim([-40, 30])

ax1.scatter(data['concentration'], data['reference_energy'],
           marker='o', label='reference')
ax1.scatter(data['concentration'], data['predicted_energy'],
           marker='x', label='CE prediction')

ax1.legend()
ax1.set_title("Full CE plot")



### ax2 = Heat of mixing plot of my own structures

ax2.set_xlabel(r'x in Li$_x$NiO2')
ax2.set_ylabel(r'Mixing energy (meV/atom)')
ax2.set_xlim([0, 1])
ax2.set_ylim([-40, 30])

# Take only the ones we are interested here
concentration    = []
referende_energy = []
predicted_energy = []

for c, ref , pred, location in zip(data["concentration"], data["reference_energy"], data["predicted_energy"], data["file_location"]):
    if "02_enumerate_P21c_0-4fu" in location:
        concentration.append(c)
        referende_energy.append(ref)
        predicted_energy.append(pred)

ax2.scatter(concentration, referende_energy,
           marker='o', label='reference')
ax2.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction')

ax2.legend()
ax2.set_title("Marcel's enumerated data")



### ax3 = Heat of mixing plot of Markus's re-relaxed_structures

ax3.set_xlabel(r'x in Li$_x$NiO2')
ax3.set_ylabel(r'Mixing energy (meV/atom)')
ax3.set_xlim([0, 1])
ax3.set_ylim([-40, 30])

# Take only the ones we are interested here
concentration    = []
referende_energy = []
predicted_energy = []

for c, ref , pred, location in zip(data["concentration"], data["reference_energy"], data["predicted_energy"], data["file_location"]):
    if "re-relax_Markus" in location:
        concentration.append(c)
        referende_energy.append(ref)
        predicted_energy.append(pred)

ax3.scatter(concentration, referende_energy,
           marker='o', label='reference')
ax3.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction')

ax3.legend()
ax3.set_title("Markus' re-relaxed structures")



### ax4 = Heat of mixing plot of the initial and final NEB images

ax4.set_xlabel(r'x in Li$_x$NiO2')
ax4.set_ylabel(r'Mixing energy (meV/atom)')
ax4.set_xlim([0, 1])
ax4.set_ylim([-40, 30])

# Take only the ones we are interested here
concentration    = []
referende_energy = []
predicted_energy = []

for c, ref , pred, location in zip(data["concentration"], data["reference_energy"], data["predicted_energy"], data["file_location"]):
    if "01_initial_structure" in location \
    or "02_odh/image"         in location \
    or "03_tsh/image"         in location \
    or "04_doube_tsh/image"   in location \
    or "0250/image"           in location \
    or "0500/image"           in location \
    or "0750/image"           in location:
        concentration.append(c)
        referende_energy.append(ref)
        predicted_energy.append(pred)

ax4.scatter(concentration, referende_energy,
           marker='o', label='reference')
ax4.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction')

ax4.legend()
ax4.set_title("Initial and final NEB images")



### ax5 = Heat of mixing plot of the NEB transition states

ax5.set_xlabel(r'x in Li$_x$NiO2')
ax5.set_ylabel(r'Mixing energy (meV/atom)')
ax5.set_xlim([0, 1])
ax5.set_ylim([-40, 30])

# Take only the ones we are interested here
concentration    = []
referende_energy = []
predicted_energy = []

for c, ref , pred, location in zip(data["concentration"], data["reference_energy"], data["predicted_energy"], data["file_location"]):
    if "/02_odh/NEB_"          in location \
    or "/03_tsh/NEB_"          in location \
    or "/04_double_tsh/NEB_"   in location \
    or "/0250/NEB_"            in location \
    or "/0500/NEB_"            in location \
    or "/0750/NEB_"            in location:
        concentration.append(c)
        referende_energy.append(ref)
        predicted_energy.append(pred)

ax5.scatter(concentration, referende_energy,
           marker='o', label='reference')
ax5.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction')

ax5.legend()
ax5.set_title("NEB transition states")



### ax6 = Check the real barriers

ax6.set_xlabel(r'x in Li$_x$NiO2')
ax6.set_ylabel(r'Barriers in eV')
ax6.set_xlim([0, 1])
#ax6.set_ylim([-40, 30])

# Front jump
concentration    = []
referende_energy = []
predicted_energy = []
for c, ref , pred, location in zip(barrier_concentrations, ref_frontjump_barriers, pred_frontjump_barriers, paths_for_training_NEB_transition_states):
    concentration.append(c)
    referende_energy.append(ref)
    predicted_energy.append(pred)
ax6.scatter(concentration, referende_energy,
           marker='o', label='reference frontjump')
ax6.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction frontjump')

# Back jump
concentration    = []
referende_energy = []
predicted_energy = []
for c, ref , pred, location in zip(barrier_concentrations, ref_backjump_barriers, pred_backjump_barriers, paths_for_training_NEB_transition_states):
    concentration.append(c)
    referende_energy.append(ref)
    predicted_energy.append(pred)
ax6.scatter(concentration, referende_energy,
           marker='o', label='reference backjump')
ax6.scatter(concentration, predicted_energy,
           marker='x', label='CE prediction backjump')


ax6.legend()
ax6.set_title("NEB transition states")





plt.savefig('/nfshome/winkelmann/ARL/tmp/mixing_energy_comparison.png', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlabel(r'reference heat of mixing [meV/atom]')
ax.set_ylabel(r'predicted heat of mixing [meV/atom]')
#ax.set_xlim([0, 1])
#ax.set_ylim([-69, 15])
ax.scatter(data['reference_energy'], data['predicted_energy'],
           marker='o', label='reference')

ax.plot(range(-30,30), range(-30,30), label='slope 1', color="red")

ax.legend()
plt.savefig('/nfshome/winkelmann/ARL/tmp/parity_plot.png', bbox_inches='tight')

In [ ]:
#data['file_location']

In [ ]:
len(file_location)

In [ ]:
#READMEs

In [ ]:
"/".join(["hello","new","world"])

In [ ]:
a = "/hello/new/world"

In [ ]:
a.split("/")

In [ ]:
#paths_for_training_NEB_transition_states

In [ ]:
#file_location

In [ ]:
data["file_location"]